## Importación de librerías

In [1]:
import pandas as pd
import os
import glob
import json
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder

## Configuración de variables globales

In [2]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
DATASETS_PATH = SETUP_JSON['datasets_path'] # Path to the datasets,
DATASETS_FOLDER = os.path.join(os.getcwd(), DATASETS_PATH) # Folder containing the datasets,
DATASETS = glob.glob(os.path.join(DATASETS_FOLDER, '*.csv')) # List of datasets
OUTPUT_CSV = SETUP_JSON['dataset_csv'] # Output CSV file
OUTPUT_PARQUET = SETUP_JSON['dataset_parquet'] # Output CSV file
TAKE_FULL_DATASET = SETUP_JSON['take_full_dataset']
N_ROWS = SETUP_JSON['n_rows']
NA_VAL = SETUP_JSON['navalues']

## Análisis de los datasets

Obtener todos los encabezados diferentes

In [3]:
unique_headers = set()
for dataset in DATASETS:
    df = pd.read_csv(dataset, nrows=1)
    df_dtypes = df.dtypes.keys()
    unique_headers.add(tuple(df_dtypes))
for header in unique_headers:
    print(header)

('Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd By

Separar los grupos de datasets por encabezados diferentes

In [4]:
datasets_by_header = {}
for header in unique_headers:
    datasets_group = []
    for dataset in DATASETS:
        df = pd.read_csv(dataset, nrows=1)
        df_dtypes = df.dtypes.keys()
        if tuple(df_dtypes) == header:
            datasets_group.append(dataset)
    datasets_by_header[header] = datasets_group
    print(f"Grupo de datasets:")
    for dataset in datasets_group:
        print(dataset)
    print(f"Número de columnas del grupo de datasets: {len(header)}")
    print()

Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv
Número de columnas del grupo de datasets: 80

Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-20-2018.csv
Número de columnas del grupo de datasets: 84



Separar los grupos de datasets por etiquetas diferentes encontradas

In [5]:
datasets_by_num_labels = {}

for datasets_group in datasets_by_header.values():
    unique_labels = set()
    for dataset in datasets_group:
        df = pd.read_csv(dataset, dtype=str)
        last_column = df.columns[-1]  # Obtiene el nombre de la última columna
        for label in list(df[last_column]):  # Usa la última columna en lugar de "Label"
            unique_labels.add(label)
    datasets_by_num_labels[tuple(datasets_group)] = len(unique_labels)
    print(f"Grupo de datasets:")
    for dataset in datasets_group:
        print(dataset)
    print(f"Etiquetas diferentes encontradas: ", unique_labels)
    print()


Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv
Etiquetas diferentes encontradas:  {'DoS attacks-SlowHTTPTest', 'Brute Force -XSS', 'Infilteration', 'Bot', 'Brute Force -Web', 'Label', 'DoS attacks-Hulk', 'Benign', 'FTP-BruteForce', 'DoS attacks-Slowloris', 'DDOS attack-HOIC', 'DDOS attack-LOIC-UDP', 'DoS attacks-Gol

Seleccionar el grupo de datasets que abarca más etiquetas diferentes

In [6]:
for datasets_group, num_labels in datasets_by_num_labels.items():
    if (num_labels) == max(datasets_by_num_labels.values()):
        datasets_selected = datasets_group
print(f"Selección de datasets a procesar:")
for dataset in datasets_selected:
    print(dataset)

Selección de datasets a procesar:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv


## Combinar datasets

Unimos todos los datasets seleccionados en un sólo dataframe

In [7]:
df = pd.DataFrame()

if TAKE_FULL_DATASET:
    for dataset in datasets_selected:
        current_df = pd.read_csv(dataset, sep=',', low_memory=False, na_values=NA_VAL)
        df = pd.concat([df, current_df], ignore_index=True)
        print(f"Dataset concatenado: {dataset}")
else:
    for dataset in datasets_selected:
        current_df = pd.read_csv(dataset, sep=',', low_memory=False, na_values=NA_VAL, nrows=N_ROWS)
        df = pd.concat([df, current_df], ignore_index=True)
        print(f"Dataset concatenado: {dataset}")

Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv


In [8]:
df.shape

(720000, 80)

In [9]:
df.dtypes

Dst Port         object
Protocol         object
Timestamp        object
Flow Duration    object
Tot Fwd Pkts     object
                  ...  
Idle Mean        object
Idle Std         object
Idle Max         object
Idle Min         object
Label            object
Length: 80, dtype: object

## Limpieza y normalización del dataframe

Eliminar encabezados repetidos

In [10]:
# Eliminar encabezados repetidos
df = df[df['Label'] != 'Label']

Eliminar valores faltantes

In [11]:
# Eliminar los valores faltantes
df = df.dropna()

In [12]:
df.shape

(718033, 80)

In [13]:
df.reset_index(drop=True, inplace=True)

Crear un json con todos los valores de etiquetas mapeados a números

In [ ]:
# Obtener los valores únicos de la columna 'Label'
unique_labels = df['Label'].unique()

# Crear un diccionario mapeando los valores únicos a números
label_mapping = {label: int(idx) for idx, label in enumerate(unique_labels)}

# Invertir el diccionario (intercambiar claves y valores)
inverted_mapping = {v: k for k, v in label_mapping.items()}

# Guardar el diccionario invertido como un archivo JSON
with open('labels.json', 'w') as json_file:
    json.dump(inverted_mapping, json_file)


print("Archivo JSON creado con el mapeo de etiquetas")

Archivo JSON creado con el mapeo de etiquetas


Convertir etiquetas a números

In [15]:
# Crear instancia del codificador
# le = LabelEncoder()
# Aplicar LabelEncoder sobre la columna 'Label'
# df['Label'] = le.fit_transform(df['Label'])

# Transformar cada valor de la columna 'Label' a su equivalente según LABELS_JSON
df['Label'] = df['Label'].map(label_mapping)

Asignar tipo correspondiente a las columnas

In [16]:
# Asignar formato decimal a columnas numéricas
for col, dtype in df.dtypes.to_dict().items():
    temp_col = df[col].dropna()
    numeric_col = pd.to_numeric(temp_col, errors='coerce').notna().all()
    if numeric_col:
        df[col] = pd.to_numeric(temp_col, errors='coerce').astype('float64')
        
# Asignar formato entero a columnas que no contienen decimales
df["Protocol"] = df["Protocol"].astype(int)
df["Dst Port"] = df["Dst Port"].astype(int)
df["Label"] = df["Label"].astype(int)

Conversión de timestamp a formato legible para el modelo

In [17]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce') # Convertimos la timestamp a formato fecha de pandas
df['Timestamp'] = np.sin(2*math.pi*df['Timestamp'].dt.hour/24)  # Convertimos la fecha en un ciclo legible para el modelo

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718033 entries, 0 to 718032
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           718033 non-null  int64  
 1   Protocol           718033 non-null  int64  
 2   Timestamp          718033 non-null  float64
 3   Flow Duration      718033 non-null  float64
 4   Tot Fwd Pkts       718033 non-null  float64
 5   Tot Bwd Pkts       718033 non-null  float64
 6   TotLen Fwd Pkts    718033 non-null  float64
 7   TotLen Bwd Pkts    718033 non-null  float64
 8   Fwd Pkt Len Max    718033 non-null  float64
 9   Fwd Pkt Len Min    718033 non-null  float64
 10  Fwd Pkt Len Mean   718033 non-null  float64
 11  Fwd Pkt Len Std    718033 non-null  float64
 12  Bwd Pkt Len Max    718033 non-null  float64
 13  Bwd Pkt Len Min    718033 non-null  float64
 14  Bwd Pkt Len Mean   718033 non-null  float64
 15  Bwd Pkt Len Std    718033 non-null  float64
 16  Fl

In [19]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,0.866025,112641719.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,56320859.5,139.300036,56320958.0,56320761.0,0
1,0,0,0.866025,112641466.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,56320733.0,114.551299,56320814.0,56320652.0,0
2,0,0,0.866025,112638623.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,56319311.5,301.934596,56319525.0,56319098.0,0
3,22,6,0.866025,6453966.0,15.0,10.0,1239.0,2273.0,744.0,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
4,22,6,0.866025,8804066.0,14.0,11.0,1143.0,2209.0,744.0,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0


In [20]:
df.tail()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
718028,8080,6,0.5,10070.0,3.0,4.0,326.0,129.0,326.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
718029,8080,6,0.5,575.0,2.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
718030,8080,6,0.5,13760.0,3.0,4.0,326.0,129.0,326.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
718031,8080,6,0.5,605.0,2.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
718032,8080,6,0.5,10255.0,3.0,4.0,326.0,129.0,326.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10


## Exportación del dataframe final en formato CSV

In [21]:
# Guardar en CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"Archivo CSV guardado como {OUTPUT_CSV}")

Archivo CSV guardado como clean_dataset.csv
